In [3]:
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

# download associations
from goatools.base import download_ncbi_associations
fin_gene2go = download_ncbi_associations()

# load ontology
from goatools.obo_parser import GODag
obodag = GODag("go-basic.obo")

# load human gene ontology
from goatools.anno.genetogo_reader import Gene2GoReader
objanno = Gene2GoReader(fin_gene2go, taxids=[9606]) #9606 is taxonomy ID for homo sapiens
ns2assoc = objanno.get_ns2assc()
for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated human genes".format(NS=nspc, N=len(id2gos)))

from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
#pop_ids = pd.read_csv('../data/df_human_geneinfo.csv',index_col=0)['GeneID'].to_list()
# df_genehumans = pd.read_csv('../data/df_human_geneinfo.csv',index_col=0)
# goeaobj = GOEnrichmentStudyNS(df_genehumans['GeneID'].to_list(), ns2assoc, obodag, propagate_counts=prop_counts, alpha=alpha, methods=[method])

# # get symbol to ID translation dictionary to get overexpressed IDs
# symbol2id = dict(zip(df_genehumans['Symbol'].str.upper(), df_genehumans['GeneID']))


  EXISTS: go-basic.obo
  EXISTS: gene2go
go-basic.obo: fmt(1.2) rel(2020-02-21) 47,245 GO Terms
HMS:0:00:04.053884 323,107 annotations, 19,649 genes, 18,246 GOs, 1 taxids READ: gene2go 
BP 17,541 annotated human genes
CC 18,648 annotated human genes
MF 17,384 annotated human genes


In [22]:
for k,v in obodag.items():
    if 'glutam' in v.name:
        print(v.id, v.name)
#obodag['GO:0048527']

GO:0000820 regulation of glutamine family amino acid metabolic process
GO:0001639 PLC activating G protein-coupled glutamate receptor activity
GO:0001640 adenylate cyclase inhibiting G protein-coupled glutamate receptor activity
GO:0001641 group II metabotropic glutamate receptor activity
GO:0001642 group III metabotropic glutamate receptor activity
GO:0002036 regulation of L-glutamate import across plasma membrane
GO:0002037 negative regulation of L-glutamate import across plasma membrane
GO:0002038 positive regulation of L-glutamate import across plasma membrane
GO:0003810 protein-glutamine gamma-glutamyltransferase activity
GO:0003839 gamma-glutamylcyclotransferase activity
GO:0003871 5-methyltetrahydropteroyltriglutamate-homocysteine S-methyltransferase activity
GO:0003922 GMP synthase (glutamine-hydrolyzing) activity
GO:0003942 N-acetyl-gamma-glutamyl-phosphate reductase activity
GO:0003952 NAD+ synthase (glutamine-hydrolyzing) activity
GO:0003991 acetylglutamate kinase activity
G

In [1]:
import pandas as pd
import numpy as np

In [9]:
# get symbol to ID translation table because I was an idiot and used the symbols for results instead of Entrez ID
# but the symbols are easier to read and figure out what they do
df_genehumans = pd.read_csv('../data/df_human_geneinfo.csv',index_col=0)['GeneID'].to_list()
symbol2id = dict(zip(df_genehumans['Symbol'].str.upper(), df_genehumans['GeneID']))


In [175]:
## this imports all gene names from human database to use for reference, but we should use only the genes that exist in the Allen database
# from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT as GeneID2nt_human # import gene ID database
# # initialize geneontology object
# goeaobj = GOEnrichmentStudyNS(
#         GeneID2nt_human.keys(), # List of all genes used in the population
#         ns2assoc, # geneid/GO associations
#         obodag, # Ontologies
#         propagate_counts = False,
#         alpha = 0.05, # default significance cut-off
#         methods = ['fdr_bh']) # defult multipletest correction method

In [16]:
# initialize geneontology object with reference as all genes in ABHA
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
goeaobj = GOEnrichmentStudyNS(symbol2id.values(), ns2assoc, obodag, propagate_counts = False, alpha = 0.05, methods = ['fdr_bh'])


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 83% 14,978 of 18,114 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 87% 15,830 of 18,114 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 83% 15,021 of 18,114 population items found in association


In [21]:
# get list of enriched genes
genes_enriched = np.squeeze(pd.read_csv('../data/go_lists/surr_corr/surr_corr_all_bf_sig.txt', header=None).values).astype(list)

# grab gene IDs from their symbols
ids_enriched  = [symbol2id[g] for g in genes_enriched]

In [22]:
# run gene ontology analysis and save significant groups
goea_results_all = goeaobj.run_study(ids_enriched)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
print('--------------------')
# print relevant info from significant
[print(go.NS, go.name, go.ratio_in_study[0]/go.ratio_in_study[1], go.p_fdr_bh) for go in goea_results_sig];


Run BP Gene Ontology Analysis: current study set of 1133 IDs ...
 84%    952 of  1,133 study items found in association
100%  1,133 of  1,133 study items found in population(18114)
Calculating 11,865 uncorrected p-values using fisher_scipy_stats
  11,865 GO terms are associated with 14,978 of 18,114 population items
   3,682 GO terms are associated with    952 of  1,133 study items
  METHOD fdr_bh:
       5 GO terms found significant (< 0.05=alpha) (  5 enriched +   0 purified): statsmodels fdr_bh
      32 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 1133 IDs ...
 89%  1,010 of  1,133 study items found in association
100%  1,133 of  1,133 study items found in population(18114)
Calculating 1,710 uncorrected p-values using fisher_scipy_stats
   1,710 GO terms are associated with 15,830 of 18,114 population items
     656 GO terms are associated with  1,010

In [1]:
from brainsmash.mapgen.stats import pearsonr, pairwise_r
from brainsmash.mapgen.stats import nonparp

# print("Spatially naive p-value:", nonparp(test_stat, naive_brainmap_corrs))
# print("SA-corrected p-value:", nonparp(test_stat, surrogate_brainmap_corrs))
# surrogate_brainmap_corrs = pearsonr(thickness, surrogate_maps).flatten()
# surrogate_pairwise_corrs = pairwise_r(surrogate_maps, flatten=True)

In [ ]:
nonparp()